<a href="https://colab.research.google.com/github/gogumalatte/face-recognizer-webapp/blob/main/%EC%95%88%EB%A9%B4%EC%9D%B8%EC%8B%9D_%EC%9B%B9_%EC%84%9C%EB%B9%84%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필수 라이브러리 설치
- `face_recognition`: 얼굴 인식 및 비교를 위한 핵심 라이브러리
- `flask`: 웹 서버 구성
- `opencv-python`: 웹캠 영상 처리 및 이미지 캡처
- `pyngrok`: Colab 환경에서 외부 공개를 위한 도구

In [1]:
# 1-1. 필수 패키지 설치
!pip install flask flask-cors mediapipe opencv-python-headless pyngrok numpy

# 1-2. ngrok 인증 토큰 설정 (ngrok.com에서 무료 계정 생성 후 토큰 발급 필요)
from pyngrok import ngrok
ngrok.set_auth_token("2wQfPVBCGGvd0mYA96vUepa8w7g_4TsNZesSbc6nvXmV38T3T")  # 발급받은 토큰으로 변경

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
grpcio-s

# 2. 프론트엔드 정적 파일을 저장할 www 디렉토리를 생성합니다.

In [2]:
!mkdir www/

# 3. 얼굴 인식을 위한 백엔드 코드를 작성합니다.
- 사용자의 기존 얼굴 이미지를 로딩하여 face_recognition으로 특징 벡터(encoding)를 추출합니다.
- 이후 웹캠 영상에서 인식된 얼굴과 비교할 기준 데이터가 됩니다.
- 클라이언트가 접속할 웹 서버를 구성하고,
- 실시간 통신(WebSocket)을 통해 얼굴 이미지를 서버로 전달받을 수 있도록 설정합니다.
- 클라이언트에서 업로드된 이미지 데이터를 메모리에서 로드하여 얼굴 인식 수행
- 등록된 얼굴과 비교 후 일치 여부(True/False)를 반환합니다.
- 클라이언트가 WebSocket을 통해 이미지 데이터를 전송하면 서버에서 비교 후 결과를 다시 전송합니다.

In [3]:
%%writefile app.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import mediapipe as mp
import cv2
import numpy as np
import base64
import io
import os
import json

app = Flask(__name__, template_folder='./www', static_folder='./www', static_url_path='/')
CORS(app)

# MediaPipe 초기화
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 데이터 저장소 (실제로는 데이터베이스 사용 권장)
face_database = {}

# 데이터 디렉토리 생성
os.makedirs('face_data', exist_ok=True)

@app.route('/')
def index():
    return send_from_directory('www', 'index.html')

def decode_image(image_data):
    # Base64 이미지 디코딩
    image_data = image_data.split(',')[1]  # 'data:image/jpeg;base64,' 부분 제거
    image_bytes = base64.b64decode(image_data)
    nparr = np.frombuffer(image_bytes, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return image

def extract_face_features(image):
    # MediaPipe로 얼굴 특징점 추출
    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
        # RGB로 변환
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 얼굴 검출
        results = face_detection.process(rgb_image)

        if results.detections:
            faces = []
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                x = int(bboxC.xmin * iw)
                y = int(bboxC.ymin * ih)
                w = int(bboxC.width * iw)
                h = int(bboxC.height * ih)

                # 얼굴 영역 크롭
                face_crop = image[y:y+h, x:x+w]

                # 얼굴 메쉬로 특징점 추출
                with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
                    face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
                    face_results = face_mesh.process(face_rgb)

                    features = []
                    if face_results.multi_face_landmarks:
                        for face_landmarks in face_results.multi_face_landmarks:
                            # 주요 특징점 추출 (예: 눈, 코, 입 등)
                            for landmark in face_landmarks.landmark:
                                features.extend([landmark.x, landmark.y, landmark.z])

                    faces.append({
                        'bbox': {'x': x, 'y': y, 'width': w, 'height': h},
                        'features': features
                    })

            return faces
    return None

def calculate_similarity(features1, features2):
    # 유클리드 거리 기반 유사도 계산
    if len(features1) != len(features2):
        return 0.0

    distance = np.linalg.norm(np.array(features1) - np.array(features2))
    # 거리를 유사도로 변환 (0~1)
    similarity = max(0, 1 - distance / 10)
    return similarity

@app.route('/register', methods=['POST'])
def register_face():
    try:
        data = request.json
        image = decode_image(data['image'])
        name = data['name']

        # 얼굴 특징 추출
        faces = extract_face_features(image)

        if not faces:
            return jsonify({
                'success': False,
                'message': '얼굴을 찾을 수 없습니다.'
            })

        # 이미 등록된 얼굴 체크
        for db_name, db_features in face_database.items():
            for face in faces:
                similarity = calculate_similarity(face['features'], db_features)
                if similarity > 0.95:
                    return jsonify({
                        'success': False,
                        'message': f'이미 등록된 얼굴입니다 (유사도: {similarity*100:.1f}%)'
                    })

        # 얼굴 등록
        face_database[name] = faces[0]['features']

        # 데이터 저장
        with open(f'face_data/{name}.json', 'w') as f:
            json.dump(faces[0]['features'], f)

        return jsonify({
            'success': True,
            'message': f'"{name}" 등록 완료'
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'message': str(e)
        })

@app.route('/recognize', methods=['POST'])
def recognize_face():
    try:
        data = request.json
        image = decode_image(data['image'])

        # 얼굴 특징 추출
        faces = extract_face_features(image)

        if not faces:
            return jsonify({
                'success': False,
                'message': '얼굴을 찾을 수 없습니다.'
            })

        result = {'faces': []}

        for face in faces:
            best_match = None
            best_similarity = 0

            for name, db_features in face_database.items():
                similarity = calculate_similarity(face['features'], db_features)
                if similarity > best_similarity and similarity > 0.7:
                    best_similarity = similarity
                    best_match = name

            face_result = {
                'x': face['bbox']['x'],
                'y': face['bbox']['y'],
                'width': face['bbox']['width'],
                'height': face['bbox']['height'],
                'name': best_match,
                'confidence': best_similarity
            }
            result['faces'].append(face_result)

        return jsonify({
            'success': True,
            'result': result
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'message': str(e)
        })

# 저장된 얼굴 데이터 로드
def load_face_data():
    global face_database
    face_data_dir = 'face_data'

    if os.path.exists(face_data_dir):
        for filename in os.listdir(face_data_dir):
            if filename.endswith('.json'):
                name = filename[:-5]  # .json 제거
                with open(os.path.join(face_data_dir, filename), 'r') as f:
                    face_database[name] = json.load(f)

# 앱 시작 시 데이터 로드
load_face_data()

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=3000, debug=True)

Writing app.py


# 4. 사용자와 상호 작용을 할 프론트엔드 코드를 작성합니다.
## HTML + JS

In [4]:
%%writefile www/index.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>안면인식 시스템</title>
    <script src="https://unpkg.com/react@17/umd/react.development.js"></script>
    <script src="https://unpkg.com/react-dom@17/umd/react-dom.development.js"></script>
    <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f0f0f0;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        video, canvas {
            max-width: 100%;
            background: #000;
            margin: 10px 0;
            border-radius: 5px;
        }
        button {
            padding: 10px 20px;
            margin: 5px;
            font-size: 16px;
            cursor: pointer;
            background: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
        }
        button:hover {
            background: #45a049;
        }
        .info-panel {
            margin-top: 20px;
            padding: 10px;
            background: #e7f3fe;
            border-left: 6px solid #2196F3;
        }
        .error {
            color: red;
            margin: 10px 0;
        }
        .success {
            color: green;
            margin: 10px 0;
        }
    </style>
</head>
<body>
    <div id="root"></div>

    <script type="text/babel">
        const FaceRecognitionApp = () => {
            const [stream, setStream] = React.useState(null);
            const [cameraType, setCameraType] = React.useState('user');
            const [message, setMessage] = React.useState('');
            const [recognitionResult, setRecognitionResult] = React.useState(null);
            const [isRegistering, setIsRegistering] = React.useState(false);
            const [personName, setPersonName] = React.useState('');

            const videoRef = React.useRef(null);
            const canvasRef = React.useRef(null);

            React.useEffect(() => {
                startCamera();
                return () => {
                    if (stream) {
                        stream.getTracks().forEach(track => track.stop());
                    }
                };
            }, [cameraType]);

            const startCamera = async () => {
                try {
                    if (stream) {
                        stream.getTracks().forEach(track => track.stop());
                    }

                    const newStream = await navigator.mediaDevices.getUserMedia({
                        video: { facingMode: cameraType },
                        audio: false
                    });

                    setStream(newStream);
                    if (videoRef.current) {
                        videoRef.current.srcObject = newStream;
                    }
                } catch (err) {
                    setMessage(`카메라 접근 오류: ${err.message}`);
                }
            };

            const toggleCamera = () => {
                setCameraType(prev => prev === 'user' ? 'environment' : 'user');
            };

            const captureImage = () => {
                const canvas = canvasRef.current;
                const video = videoRef.current;

                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;

                const context = canvas.getContext('2d');
                context.drawImage(video, 0, 0);

                return canvas.toDataURL('image/jpeg', 0.8);
            };

            const registerFace = async () => {
                if (!personName.trim()) {
                    setMessage('이름을 입력해주세요.');
                    return;
                }

                try {
                    setIsRegistering(true);
                    const imageData = captureImage();

                    const response = await fetch('/register', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                        },
                        body: JSON.stringify({
                            image: imageData,
                            name: personName
                        }),
                    });

                    const data = await response.json();

                    if (data.success) {
                        setMessage(`안면 등록 성공: ${data.message}`);
                        setPersonName('');
                    } else {
                        setMessage(`안면 등록 실패: ${data.message}`);
                    }
                } catch (error) {
                    setMessage(`등록 오류: ${error.message}`);
                } finally {
                    setIsRegistering(false);
                }
            };

            const recognizeFace = async () => {
                try {
                    const imageData = captureImage();

                    const response = await fetch('/recognize', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                        },
                        body: JSON.stringify({
                            image: imageData
                        }),
                    });

                    const data = await response.json();

                    if (data.success) {
                        setRecognitionResult(data.result);
                        drawBoundingBox(data.result);
                        setMessage('안면 인식 성공!');
                    } else {
                        setMessage(`안면 인식 실패: ${data.message}`);
                        setRecognitionResult(null);
                    }
                } catch (error) {
                    setMessage(`인식 오류: ${error.message}`);
                }
            };

            const drawBoundingBox = (result) => {
                const canvas = canvasRef.current;
                const video = videoRef.current;
                const context = canvas.getContext('2d');

                // 원본 이미지 다시 그리기
                context.drawImage(video, 0, 0, canvas.width, canvas.height);

                // 바운딩 박스 그리기
                if (result.faces && result.faces.length > 0) {
                    result.faces.forEach(face => {
                        context.strokeStyle = '#00ff00';
                        context.lineWidth = 3;
                        context.strokeRect(face.x, face.y, face.width, face.height);

                        // 이름 표시
                        if (face.name) {
                            context.fillStyle = '#00ff00';
                            context.font = '20px Arial';
                            context.fillText(face.name, face.x, face.y - 5);
                        }
                    });
                }
            };

            return (
                <div className="container">
                    <h1>안면인식 시스템</h1>

                    <div>
                        <video
                            ref={videoRef}
                            autoPlay
                            playsInline
                            style={{ display: 'block' }}
                        />
                        <canvas
                            ref={canvasRef}
                            style={{ display: 'none' }}
                        />
                    </div>

                    <div>
                        <button onClick={toggleCamera}>
                            카메라 전환 ({cameraType === 'user' ? '전면' : '후면'})
                        </button>
                    </div>

                    <div>
                        <h3>안면 등록</h3>
                        <input
                            type="text"
                            value={personName}
                            onChange={(e) => setPersonName(e.target.value)}
                            placeholder="이름 입력"
                            style={{ padding: '10px', marginRight: '10px' }}
                        />
                        <button
                            onClick={registerFace}
                            disabled={isRegistering}
                        >
                            {isRegistering ? '등록 중...' : '안면 등록'}
                        </button>
                    </div>

                    <div>
                        <button onClick={recognizeFace}>
                            안면 인식
                        </button>
                    </div>

                    {message && (
                        <div className={message.includes('성공') ? 'success' : 'error'}>
                            {message}
                        </div>
                    )}

                    {recognitionResult && recognitionResult.faces && (
                        <div className="info-panel">
                            <h3>인식 결과</h3>
                            {recognitionResult.faces.map((face, index) => (
                                <div key={index}>
                                    <p>이름: {face.name || '알 수 없음'}</p>
                                    <p>신뢰도: {(face.confidence * 100).toFixed(1)}%</p>
                                    <p>위치: ({face.x}, {face.y})</p>
                                    <p>크기: {face.width}x{face.height}</p>
                                </div>
                            ))}
                        </div>
                    )}
                </div>
            );
        };

        ReactDOM.render(<FaceRecognitionApp />, document.getElementById('root'));
    </script>
</body>
</html>

Writing www/index.html


# 5. 한 번에 위 서버를 실행하고 접근 URL을 여는 실행 스크립트를 작성합니다.

In [6]:
%%writefile run_server.py
import subprocess
import time
from pyngrok import ngrok

# Flask 서버 시작
server_process = subprocess.Popen(["python", "app.py"])
print("Flask 서버가 시작되었습니다.")

# ngrok 터널 생성
http_tunnel = ngrok.connect(3000)
print(f"ngrok 터널이 생성되었습니다: {http_tunnel.public_url}")
print(f"모바일에서 접속하세요: {http_tunnel.public_url}")

try:
    # 앱이 계속 실행되도록 대기
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    # 종료 시 프로세스 정리
    server_process.terminate()
    ngrok.kill()

Overwriting run_server.py


# 6. 5단계에서 생성한 스크립트를 실행합니다. (서버를 실행)

In [7]:
# 서버 실행
!python run_server.py

Flask 서버가 시작되었습니다.
ngrok 터널이 생성되었습니다: https://3379-34-125-46-233.ngrok-free.app
모바일에서 접속하세요: https://3379-34-125-46-233.ngrok-free.app
2025-05-19 11:31:02.232205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747654262.264696    2895 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747654262.274529    2895 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 11:31:02.304623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriat

In [8]:
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (84.8 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [9]:
!tree

.
├── app.py
├── face_data
│   └── 최기영.json
├── run_server.py
├── sample_data
│   ├── anscombe.json
│   ├── california_housing_test.csv
│   ├── california_housing_train.csv
│   ├── mnist_test.csv
│   ├── mnist_train_small.csv
│   └── README.md
└── www
    └── index.html

3 directories, 10 files
